# 🚀 Advanced RAG (Retrieval-Augmented Generation)

## 📘 개요
**Advanced RAG**는 기본 RAG보다 한 단계 발전된 구조로,  
단순히 “검색 → 문맥 삽입 → 답변 생성”의 흐름을 넘어  
**정확도**, **효율성**, **확장성**을 개선하기 위한 기술들이 결합된 형태입니다.

---

## 🧩 기본 RAG 복습

### 🔹 기본 구조
1. **Query Embedding**: 사용자의 질문을 벡터로 변환  
2. **Vector Search**: 벡터 DB에서 유사 문서 검색  
3. **Context Injection**: 검색된 텍스트를 프롬프트에 추가  
4. **LLM Generation**: LLM이 답변 생성

### ⚠️ 한계점
- 검색된 문서 중 진짜 관련 정보만 선별하기 어려움  
- 검색 쿼리가 모호할 경우 적절한 문서가 검색되지 않음  
- LLM의 답변이 실제 근거 문서와 다를 수 있음 (hallucination)  
- 긴 문서나 다중 hop 질의에 대응하기 어려움

---

## 🚀 Advanced RAG의 주요 구성요소

### 1️⃣ Query Rewriting (질문 개선)
- LLM이 사용자의 질문을 **검색 친화적으로 재작성**
- 예시  
  - 사용자: “그 회사 매출 어땠어?”  
  - 변환: “2023년 삼성전자의 연간 매출액”
- LangChain 예시: `MultiQueryRetriever`, `ContextualCompressionRetriever`

---

### 2️⃣ Re-ranking (재정렬)
- 벡터 검색 결과를 LLM 또는 Cross-Encoder로 재평가하여  
  **가장 관련도가 높은 문서만 남김**
- 예시:  
  - 상위 20개 문서 → LLM 기반 재평가 → 상위 5개 사용
- 도구 예시: `bge-reranker`, `Cohere Rerank API`, LangChain `ReRanker`

---

### 3️⃣ Context Compression / Summarization
- 너무 긴 문서를 그대로 LLM에 넣지 않고  
  **요약(summarization)** 또는 **핵심 문장만 추출(extractive compression)**  
- LangChain: `ContextualCompressionRetriever`, `LLMChainExtractor`

---

### 4️⃣ Multi-hop Retrieval
- 한 질문을 여러 하위 질문으로 나누어 **단계적 검색** 수행  
- 예시:  
  1. “AI 반도체 주요 기업은?”  
  2. “NVIDIA의 매출 구조는?”
- 대표 기법: **Self-Ask**, **ReAct**, **Chain-of-Thought RAG**

---

### 5️⃣ Hybrid Retrieval (혼합 검색)
- **벡터 검색 + 키워드 검색 + 메타데이터 필터링**을 결합  
- 예시:  
  `vector + keyword + date filter`
- 장점: 의미적 유사성과 시기적 제약을 함께 반영  
- Milvus: `hybrid_search()` 기능 활용 가능

---

### 6️⃣ Structured Retrieval / Graph RAG
- 문서 간 관계를 **그래프(Neo4j, ArangoDB 등)** 로 저장  
- LLM이 그래프 탐색을 통해 의미적 연결성을 찾아냄  
- 예시:  
  - 논문 인용 관계, FAQ 연결 구조 등  
- 장점: 논리적 reasoning과 근거 추적에 강함

---

### 7️⃣ Response Verification & Grounding
- LLM이 생성한 답변을 **다시 검증**하여 실제 근거 문서와 일치하는지 확인  
- 방법  
  - LLM이 스스로 “이 답변이 문서 내용과 일치합니까?” 재질문  
  - 출처(Citation) 자동 추가  
  - Self-RAG (Meta, 2024): 모델이 자체적으로 “retrieve → answer → verify → revise” 수행

---

### 8️⃣ Caching / Memory / Feedback Loop
- 과거의 질의응답을 저장하여 **속도 향상 및 재사용**  
- 오답 시 LLM이 피드백을 학습하도록 구성  
- 예시: `LangChain Memory`, `VectorCache`, `User Feedback Loop`

---

## ⚙️ Advanced RAG 파이프라인 예시

```text
사용자 질문
   ↓
**질문 재작성 (Query Rewriter)** 
   ↓
**Re-ranking (LLM 또는 Cross-Encoder)**
   ↓
Context Summarization (압축)
   ↓
LLM 답변 생성
   ↓
Answer Verification (Self-RAG)
   ↓
결과 + 출처 반환

In [3]:
import os
from pathlib import Path
from dotenv import load_dotenv

from pymilvus import MilvusClient
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

# ── 설정 ──
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("OPENAI_API_KEY 환경 변수가 필요합니다.")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1, openai_api_key=OPENAI_API_KEY)
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)

# Milvus 연결
db_dir = '/Users/n-jison/Desktop/jaeig/kmu-agent-course/AI_Agent/retrieve'
db_name = 'kmu.db'
db_url = os.path.join(db_dir, db_name)
client = MilvusClient(db_url)

# 연결 확인
collections = client.list_collections()
print("Milvus Collections:", collections)
stats = client.get_collection_stats(collection_name='kmu_llm')
print("Collection stats:", stats)


Milvus Collections: ['kmu_llm']
Collection stats: {'row_count': 206}


In [4]:
## Advanced RAG 파이프라인 구현
def rewrite_question(question: str) -> str:
    """
    Query Rewriter: 질문을 검색에 유리한 형태로 변환
    (예: 맥락을 보강하거나 키워드 중심으로 재구성)
    """
    prompt = ChatPromptTemplate.from_template(
        "사용자의 질문을 검색에 유리하게 재작성해 주세요.\n"
        "원본 질문: {question}\n"
        "재작성된 질문:"
    )
    chain = prompt | llm | StrOutputParser()
    rewritten = chain.invoke({"question": question})
    return rewritten.strip()


def milvus_search(question_embedding, top_k: int = 5):
    """
    Milvus 검색, 검색 결과 리스트 반환
    """
    res = client.search(
        collection_name='kmu_llm',
        data=[question_embedding],
        limit=top_k,
        output_fields=['id', 'text', 'meta']
    )
    if not res or len(res[0]) == 0:
        return []
    return res[0]  # HybridHits 리스트처럼 동작


def rank_results(results) -> list:
    """
    Re-ranking: 검색된 결과를 LLM을 이용해 재정렬
    여기서는 간단히 LLM에게 순위를 다시 매기도록 요청
    """
    # 결과를 텍스트와 ID 리스트로 만들기
    candidates = []
    for hit in results:
        ent = hit.entity
        meta = getattr(ent, "meta", {})
        text = getattr(ent, "text", "")
        source = meta.get("source", "")
        candidates.append({"text": text, "source": source})

    # LLM에게 순위 매기기 요청
    prompt = ChatPromptTemplate.from_template(
        "다음 문서들을 질문과 관련한 순서대로 재정렬하세요.\n\n"
        "문서들:\n"
        "{docs}\n\n"
        "재정렬된 순서의 인덱스들을 쉼표로 나열해서 주세요 (예: 2,1,3):"
    )
    docs_str = "\n\n".join(f"{i+1}. ({c['source']}) {c['text'][:200]}" for i, c in enumerate(candidates))
    chain = prompt | llm | StrOutputParser()
    order = chain.invoke({"docs": docs_str}).strip()
    # order 예: "2,1,3"
    try:
        idxs = [int(x) - 1 for x in order.split(",")] 
    except:
        # 실패 시 원래 순서 유지
        idxs = list(range(len(candidates)))
    reordered = [candidates[i] for i in idxs if 0 <= i < len(candidates)]
    return reordered


def compress_context(ranked_docs: list, max_total_chars: int = 1500) -> str:
    """
    컨텍스트 압축: 여러 문서를 하나의 문자열로 합치되, 길이 제한 고려
    """
    parts = []
    total = 0
    for doc in ranked_docs:
        text = doc["text"]
        src = doc["source"]
        entry = f"[{src}]\n{text}\n"
        l = len(entry)
        if total + l > max_total_chars:
            break
        parts.append(entry)
        total += l
    return "\n---\n".join(parts)


def verify_answer(question: str, answer: str, context: str) -> bool:
    """
    Self-verification: LLM에게 답변이 컨텍스트에 근거했는지 확인 요청
    """
    prompt = ChatPromptTemplate.from_template(
        "아래는 질문, 답변, 그리고 문맥입니다.\n"
        "답변이 문맥에 근거한 것인지 검증해 주세요.\n"
        "만약 근거가 없다면 'No'를, 근거가 있으면 'Yes'를 출력하세요.\n\n"
        "질문: {question}\n"
        "답변: {answer}\n"
        "문맥:\n{context}\n\n"
        "검증 결과:"
    )
    chain = prompt | llm | StrOutputParser()
    res = chain.invoke({"question": question, "answer": answer, "context": context}).strip().lower()
    return res.startswith("yes")



In [5]:

def advanced_rag(question: str, top_k: int = 5) -> str:
    """
    Advanced RAG 파이프라인
    """
    # 1) 질문 재작성
    rewritten = rewrite_question(question)

    # 2) 재작성된 질문 임베딩
    q_emb = embedding_model.embed_query(rewritten)

    # 3) Milvus 검색
    results = milvus_search(q_emb, top_k=top_k)

    # 4) Re-ranking
    ranked = rank_results(results)

    # 5) 컨텍스트 압축
    context = compress_context(ranked)

    # 6) 답변 생성
    prompt_template = ChatPromptTemplate.from_template(
        "다음 문맥을 참고하여 질문에 답하세요.\n\n"
        "질문: {question}\n"
        "문맥:\n{context}\n\n"
        "답변:"
    )
    chain = prompt_template | llm | StrOutputParser()
    answer = chain.invoke({"question": question, "context": context})

    # 7) 검증
    ok = verify_answer(question, answer, context)
    if not ok:
        return "죄송합니다. 제공된 문서로는 충분한 정보를 찾을 수 없습니다."

    # 8) 최종 반환 (출처 포함)
    # 출처: ranked 문서의 source 리스트
    sources = [doc["source"] for doc in ranked]
    unique_sources = list(dict.fromkeys(sources))
    return f"{answer}\n\n출처: {', '.join(unique_sources)}"

In [6]:
question = "경영정보학과 취득 가능 자격증 종류는 무엇인가요"
res = advanced_rag(question, top_k=5)
print("질문:", question)
print("답변:", res)

질문: 경영정보학과 취득 가능 자격증 종류는 무엇인가요
답변: 경영정보학과에서 취득 가능한 자격증 종류는 다음과 같습니다:

1. 정보처리기사
2. 빅데이터분석기사
3. SQL개발자
4. 데이터분석준전문가
5. 경영정보시각화능력

이 자격증을 제출하면 졸업논문이 면제됩니다.

출처: /Users/n-jison/Desktop/jaeig/kmu-agent-course/AI_Agent/retrieve/files/2025_bigdata_bluebook.pdf, /Users/n-jison/Desktop/jaeig/kmu-agent-course/AI_Agent/retrieve/files/2025_mis_bluebook.pdf
